In [1]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("rahimanshu/cardiomegaly-disease-prediction-using-cnn")

print("Path to dataset files:", path)

100%|██████████| 61.7M/61.7M [00:00<00:00, 78.9MB/s]

Extracting files...


Path to dataset files: /root/.cache/kagglehub/datasets/rahimanshu/cardiomegaly-disease-prediction-using-cnn/versions/1


In [2]:
import os
from pathlib import Path

import torch
from PIL import Image
from torch.utils.data import Dataset, DataLoader
from torchvision.models import EfficientNet_B5_Weights, EfficientNet_B3_Weights, EfficientNet_V2_S_Weights, \
    ResNet50_Weights, EfficientNet_B7_Weights, EfficientNet_B6_Weights
from torchvision.transforms import Compose, Resize, ToTensor, Normalize, RandomHorizontalFlip, RandomRotation

import torch.nn as nn
from torch import optim
from tqdm import tqdm

import torchvision
!pip install efficientnet_pytorch
!pip install torchinfo
from efficientnet_pytorch import EfficientNet
from torchinfo import summary

### Датасет
class CardiomegalyDataset(Dataset):
    def __init__(self, data_dir: str, recursive=True):
        """
        Инициализирует набор данных кардиомегалии с возможностью рекурсивного поиска классов.
        :param data_dir: Основной путь к директории с изображениями ('train' or 'test')
        :param recursive: Рекурсивно искать классы в подпапках (default: True)
        """
        self.transform = EfficientNet_B3_Weights.IMAGENET1K_V1.transforms()

        # Найдем все папки 'true' и 'false'
        root_path = Path(data_dir)
        if recursive: # Рекурсивно ищем папки 'true' и 'false'
            true_folders = list(root_path.rglob("true"))
            false_folders = list(root_path.rglob("false"))
        else: # Если не рекурсивно, ограничимся первым уровнем вложенности
            true_folders = [Path(root_path / "true")]
            false_folders = [Path(root_path / "false")]

        # Собираем полные пути к файлам и соответствующим меткам
        paths_and_labels = []
        for folder in true_folders:
            files_in_true = sorted(folder.iterdir())
            paths_and_labels.extend([(file, True) for file in files_in_true])

        for folder in false_folders:
            files_in_false = sorted(folder.iterdir())
            paths_and_labels.extend([(file, False) for file in files_in_false])

        self.paths_and_labels = paths_and_labels

    def __len__(self):
        return len(self.paths_and_labels)

    def __getitem__(self, idx):
        path, label = self.paths_and_labels[idx]
        img = Image.open(path).convert('RGB')
        transformed_img = self.transform(img)
        target = torch.tensor(int(label))
        return transformed_img, target

def get_cardiomegaly_dataloaders(
    train_data_path: str,
    test_data_path: str,
    batch_size: int = 16,
    num_workers: int = 4,
    shuffle_train: bool = True,
    recursive: bool = True
):
    """
    Создаем загрузчики данных для тренировочных и тестовых данных, используя CardiomegalyDataset.
    :param train_data_path: Путь к тренировочным данным
    :param test_data_path: Путь к тестовым данным
    :param batch_size: Размер батчей
    :param num_workers: Число потоков для параллельной загрузки данных
    :param shuffle_train: Нужно ли перемешивать тренировочные данные
    :param recursive: Флаг, разрешающий рекурсивный поиск классов (по умолчанию включен)
    :return: Кортеж (train_loader, test_loader) """

    train_dataset = CardiomegalyDataset(train_data_path, recursive=recursive)
    test_dataset = CardiomegalyDataset(test_data_path, recursive=recursive)

    # DataLoader для тренировок
    train_loader = DataLoader(
        dataset=train_dataset,
        batch_size=batch_size,
        shuffle=shuffle_train,
        num_workers=num_workers
    )

    # DataLoader для тестов
    test_loader = DataLoader(
        dataset=test_dataset,
        batch_size=batch_size,
        shuffle=False,
        num_workers=num_workers
    )

    return train_loader, test_loader

### Модель
class CardiomegalyClassifier(nn.Module):
    def __init__(self, num_classes=2):
        super().__init__()
        self.base_model = EfficientNet.from_pretrained('efficientnet-b3') # num_classes=2
        self.head = nn.Sequential(
            nn.BatchNorm1d(self.base_model._fc.in_features),
            nn.Linear(self.base_model._fc.in_features, 256),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(256, num_classes),
            nn.LogSoftmax(dim=1)  # Логарифмическая Softmax для кросс-энтропии
        )

    def forward(self, x):
        features = self.base_model.extract_features(x) # Через EfficientNet получаем признаки
        pooled_features = nn.functional.adaptive_max_pool2d(features, 1) # Применяем глобальный max pooling
        flattened_features = pooled_features.view(pooled_features.size(0), -1)
        output = self.head(flattened_features) # Прогоняем через головной слой
        return output

### Обучение
def run_epoch(model, data_loader, loss_fn, optimizer=None, device=None, is_test=False):
    """ Выполняет одну эпоху обучения или валидации.
    :param model: Инстанс PyTorch модели
    :param data_loader: DataLoader (для обучения или тестирования)
    :param loss_fn: Критерий потерь (например, BCELoss)
    :param optimizer: Оптимизатор (только для режима обучения)
    :param device: Устройство (CPU/GPU)
    :param is_test: Флаг, определяющий режим (training vs testing)
    :return: Средняя потеря и точность за эпоху """
    if device is None:
        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    if is_test:
        model.eval()
    else:
        model.train()

    total_loss = 0
    total_correct = 0
    total_samples = 0

    with torch.set_grad_enabled(not is_test):
        progress_bar = tqdm(data_loader, leave=False)
        for inputs, labels in progress_bar:
            inputs, labels = inputs.to(device), labels.to(device)

            if not is_test:
                optimizer.zero_grad()

            outputs = model.forward(inputs)
            loss = loss_fn(outputs, labels)

            if not is_test:
                loss.backward()
                optimizer.step()

            total_loss += loss.item() # * inputs.size(0)
            pred = outputs.argmax(dim=1)
            total_correct += (pred == labels).float().sum().item()
            total_samples += inputs.size(0)

            progress_bar.set_description(
                f"Loss: {total_loss / total_samples:.4f}, Acc: {total_correct / total_samples:.4f}")

    return total_loss / len(data_loader), total_correct / total_samples


def train_model(model, train_loader, test_loader, epochs=10, lr=0.001, device=None, optimizer=None):
    """ Управляет процессом обучения и периодической проверкой модели.
    :param model: Инстанс PyTorch модели
    :param train_loader: DataLoader для обучения
    :param test_loader: DataLoader для тестирования
    :param epochs: Количество эпох обучения
    :param lr: Скорость обучения
    :param device: Устройство (CPU/GPU)
    :param optimizer: Пользовательский оптимизатор (опциональный)
    :return: Словарь метрик обучения и тестирования """
    if device is None:
        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    if optimizer is None:
        optimizer = torch.optim.Adamax(model.parameters(), lr=lr, weight_decay=0.0001) # Adam

    model = model.to(device)

    loss_fn = nn.CrossEntropyLoss()
    lr_scheduler = optim.lr_scheduler.MultiStepLR(optimizer=optimizer,
                                                  milestones=[int(epochs * 0.5), int(epochs * 0.75)], gamma=0.1,
                                                  last_epoch=-1)
    metrics = {"train_loss": [], "train_acc": [], "val_loss": [], "val_acc": []}

    best_test_loss = float('inf')
    best_test_acc = 0

    for epoch in range(epochs):
        print(f"Epoch {epoch + 1}/{epochs}:")

        train_loss, train_acc = run_epoch(model, train_loader, loss_fn, optimizer, device, is_test=False)
        metrics["train_loss"].append(train_loss)
        metrics["train_acc"].append(train_acc)
        print(f"Train Loss: {train_loss:.4f}, Train Accuracy: {train_acc:.4f}")

        val_loss, val_acc = run_epoch(model, test_loader, loss_fn, device=device, is_test=True)
        metrics["val_loss"].append(val_loss)
        metrics["val_acc"].append(val_acc)
        print(f"Validation Loss: {val_loss:.4f}, Validation Accuracy: {val_acc:.4f}\n")

        lr_scheduler.step()

    return metrics

if __name__ == "__main__":
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    train_data_path = '/root/.cache/kagglehub/datasets/rahimanshu/cardiomegaly-disease-prediction-using-cnn/versions/1/train/train'
    test_data_path = '/root/.cache/kagglehub/datasets/rahimanshu/cardiomegaly-disease-prediction-using-cnn/versions/1/test/test'

    model = CardiomegalyClassifier()
    # model = EfficientNet.from_pretrained('efficientnet-b3', num_classes=2)

    train_loader, test_loader = get_cardiomegaly_dataloaders(train_data_path, test_data_path, recursive=False)

    for inputs, labels in train_loader:
        print(labels[:5])
        print(inputs.shape, labels.shape)
        summary(model, input_size=inputs.shape)
        break

    metrics = train_model(model, train_loader, test_loader, epochs=10, lr=0.0005)

Loaded pretrained weights for efficientnet-b3


/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py:624: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(


tensor([0, 0, 0, 1, 1])
torch.Size([16, 3, 300, 300]) torch.Size([16])
Epoch 1/10:


Train Loss: 0.5940, Train Accuracy: 0.6749


Validation Loss: 0.5060, Validation Accuracy: 0.7558

Epoch 2/10:


...

Validation Loss: 0.6652, Validation Accuracy: 0.8061

Epoch 9/10:


Train Loss: 0.0565, Train Accuracy: 0.9826


Validation Loss: 0.6552, Validation Accuracy: 0.8079

Epoch 10/10:


Train Loss: 0.0512, Train Accuracy: 0.9829


Validation Loss: 0.6686, Validation Accuracy: 0.8079



In [ ]:
import numpy as np
import pandas as pd

import os
from PIL import Image

import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import confusion_matrix , classification_report

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D , MaxPooling2D , Flatten , Activation , Dense , Dropout , BatchNormalization
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam , Adamax
from tensorflow.keras import regularizers

### Датасет
def get_data(data_path):
  filepaths = []
  labels = []

  for fold in os.listdir(train_data_path):
      f_path = os.path.join(train_data_path , fold)
      filelists = os.listdir(f_path)
      for file in filelists:
          filepaths.append(os.path.join(f_path , file))
          labels.append(fold)

  return pd.concat([pd.Series(filepaths , name='filepaths') , pd.Series(labels , name='label')] , axis=1)

train_data_path = '/root/.cache/kagglehub/datasets/rahimanshu/cardiomegaly-disease-prediction-using-cnn/versions/1/train/train'
test_data_path = '/root/.cache/kagglehub/datasets/rahimanshu/cardiomegaly-disease-prediction-using-cnn/versions/1//test/test'

train_df = get_data(train_data_path)
test_df = get_data(test_data_path)

img_size = (224,224)
batch_size = 16

train_loader = ImageDataGenerator().flow_from_dataframe(train_df,
  x_col='filepaths',
  y_col='label',
  target_size=img_size,
  class_mode='categorical',
  color_mode='rgb',
  shuffle=True,
  batch_size=batch_size)


test_loader = ImageDataGenerator().flow_from_dataframe(test_df,
  x_col='filepaths',
  y_col='label' ,
  target_size=img_size,
  class_mode='categorical',
  color_mode='rgb',
  shuffle=False ,
  batch_size=batch_size)

### Модель
num_class = len(train_loader.class_indices)
base_model = tf.keras.applications.EfficientNetB3(include_top=False, weights='imagenet', input_shape=(img_size[0], img_size[1], 3), pooling='max')

model = Sequential([
    base_model,
    BatchNormalization(),
    Dense(256, activation='relu'),
    Dropout(0.2),
    Dense(num_class, activation='softmax')
])
model.compile(optimizer=Adamax(learning_rate=0.0005), loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

### Обучение
history = model.fit(x=train_loader, epochs=10 , verbose=1 , validation_data=test_loader)

Found 4438 validated image filenames belonging to 2 classes.
Found 4438 validated image filenames belonging to 2 classes.
43941136/43941136 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ efficientnetb3 (Functional)     │ (None, 1536)           │    10,783,535 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 1536)           │         6,144 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 256)            │       393,472 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 2)              │           514 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 11,183,665 (42.66 MB)

 Trainable params: 11,093,290 (42.32 MB)

 Non-trainable params: 90,375 (353.03 KB)

Epoch 1/10


/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


278/278 ━━━━━━━━━━━━━━━━━━━━ 233s 420ms/step - accuracy: 0.5679 - loss: 0.9052 - val_accuracy: 0.6848 - val_loss: 0.6249
Epoch 2/10
278/278 ━━━━━━━━━━━━━━━━━━━━ 49s 178ms/step - accuracy: 0.7325 - loss: 0.5919 - val_accuracy: 0.8272 - val_loss: 0.3891
Epoch 3/10
278/278 ━━━━━━━━━━━━━━━━━━━━ 82s 178ms/step - accuracy: 0.8093 - loss: 0.4146 - val_accuracy: 0.8914 - val_loss: 0.2603
Epoch 4/10
278/278 ━━━━━━━━━━━━━━━━━━━━ 60s 217ms/step - accuracy: 0.8520 - loss: 0.3534 - val_accuracy: 0.9362 - val_loss: 0.1585
Epoch 5/10
278/278 ━━━━━━━━━━━━━━━━━━━━ 61s 218ms/step - accuracy: 0.8858 - loss: 0.2700 - val_accuracy: 0.9691 - val_loss: 0.0901
Epoch 6/10
278/278 ━━━━━━━━━━━━━━━━━━━━ 50s 181ms/step - accuracy: 0.9170 - loss: 0.2041 - val_accuracy: 0.9811 - val_loss: 0.0571
Epoch 7/10
278/278 ━━━━━━━━━━━━━━━━━━━━ 51s 183ms/step - accuracy: 0.9484 - loss: 0.1401 - val_accuracy: 0.9944 - val_loss: 0.0264
Epoch 8/10
278/278 ━━━━━━━━━━━━━━━━━━━━ 51s 183ms/step - accuracy: 0.9462 - loss: 0.1332 - va